# Semi-supervised - label propagation

In [3]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
import statistics
from sklearn.multiclass import OneVsRestClassifier
import time

data=pd.read_csv(r'C:\Users\ADMIN\Desktop\new_df.csv')
df = pd.DataFrame(data)
test_df = df

print(df.shape)


(67836, 22)


In [4]:
label_encoder = LabelEncoder()
df['readmitted']= label_encoder.fit_transform(df['readmitted']) 
print(df['readmitted'].shape)
print(df['readmitted'].head(5))

(67836,)
0    1
1    2
2    2
3    2
4    1
Name: readmitted, dtype: int32


In [5]:
y = data['readmitted']
x = data.drop('readmitted', axis=1)
print('target : ', y.shape)
print('features : ',x.shape)

target :  (67836,)
features :  (67836, 21)


In [6]:
x = pd.get_dummies(x,drop_first=True)
print(x.shape)




(67836, 36)


In [7]:
# user-defined unlabelling function

import random

def maskfunc(true_target, percentage):
    if (percentage >=0 and percentage <= 100):
       
        n_total_samples = len(true_target)
        unlabel = (100-percentage)
        n_labeled_points = int((unlabel * n_total_samples)/100)
        indices = np.arange(n_total_samples)
        unlabeled_set = indices[n_labeled_points:]
        
        labels = true_target.copy()
        labels[unlabeled_set] = -1
        
        print('Total samples considered :\t\t',n_total_samples)
        print('Count of labelled points at ',percentage,'% :\t',n_labeled_points)
        print('Count of unlabelled points at ',percentage,'% :\t',(n_total_samples - n_labeled_points))
        return labels,unlabeled_set
    else: 
        return 'Percentage not in range of (0,100)'
    


In [10]:
# classification report and confusion matrix function
from sklearn.metrics import roc_curve,auc,precision_recall_curve,confusion_matrix,accuracy_score,classification_report

def report_conf(unlabeled_set_value, percen):
    unlabeled_set = unlabeled_set_value
    predicted_labels = label_prop_model.transduction_[unlabeled_set]
    true_labels = Y[unlabeled_set]
    
    cm = confusion_matrix(true_labels, predicted_labels, labels=label_prop_model.classes_)
    print('Classifcation report at ',percen,'% :\n')
    print(classification_report(true_labels, predicted_labels))

    print('Confusion matrix at ',percen,'% :\n',cm)

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation

score = {}
#RUN THE MODEL
start = time.time()
label_prop_model = LabelPropagation(kernel='knn', n_neighbors=8, max_iter=100, gamma=20,n_jobs=None)

# 100% labelled data
ml,unlabeled_set_0 = maskfunc(y, 0)
label_prop_model.fit(x, ml)
score_zero = label_prop_model.score(x, ml)
print('Accuracy with 0 % unlabelled data\t',round(score_zero,4))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')



Total samples considered : 67836
Count of labelled points at  0 % : 67836
Count of unlabelled points at  0 % : 0

Accuracy with 0 % unlabelled data 0.6457


In [83]:
# 90% labelled data
start = time.time()
ml,unlabeled_set_10 = maskfunc(y, 10)
label_prop_model.fit(x, ml)
score_ten = label_prop_model.score(x, ml)
print('\nAccuracy with 10 % unlabelled data\t',round(score_ten,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf(unlabeled_set_10,10)


Total samples considered :		 67836
Count of labelled points at  10 % :	 61052
Count of unlabelled points at  10 % :	 6784
Accuracy with 10 % unlabelled data	 0.58

Metrics:

Classifcation report at 0% :

              precision    recall  f1-score   support

           0       0.07      0.02      0.04       603
           1       0.19      0.32      0.24      1238
           2       0.73      0.67      0.70      4943

    accuracy                           0.55      6784
   macro avg       0.33      0.34      0.32      6784
weighted avg       0.57      0.55      0.56      6784

Confusion matrix at 0% :
 [[  15  168  420]
 [  42  390  806]
 [ 166 1463 3314]]


In [84]:
# 80% labelled data
start = time.time()
ml,unlabeled_set_20 = maskfunc(y, 20)
label_prop_model.fit(x, ml)
score_twenty = label_prop_model.score(x, ml)
print('\nAccuracy with 20 % unlabelled data\t',round(score_twenty,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf(unlabeled_set_20,20)

Total samples considered :		 67836
Count of labelled points at  20 % :	 54268
Count of unlabelled points at  20 % :	 13568
Accuracy with 20 % unlabelled data	 0.5
Classifcation report at  20 % :

              precision    recall  f1-score   support

           0       0.09      0.16      0.12      1115
           1       0.23      0.30      0.26      3029
           2       0.69      0.56      0.62      9424

    accuracy                           0.47     13568
   macro avg       0.34      0.34      0.33     13568
weighted avg       0.54      0.47      0.49     13568

Confusion matrix at  20 % :
 [[ 180  294  641]
 [ 439  904 1686]
 [1386 2805 5233]]


In [85]:
# 50% labelled data
start = time.time()
ml,unlabeled_set_50 = maskfunc(y, 50)
label_prop_model.fit(x, ml)
score_fifty = label_prop_model.score(x, ml)
print('\nAccuracy with 50 % unlabelled data\t',round(score_fifty,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf(unlabeled_set_50,50)


Total samples considered :		 67836
Count of labelled points at  50 % :	 33918
Count of unlabelled points at  50 % :	 33918
Accuracy with 50 % unlabelled data	 0.19

Metrics:

Classifcation report at  50 % :

              precision    recall  f1-score   support

           0       0.09      0.78      0.15      2858
           1       0.31      0.08      0.13     10313
           2       0.60      0.14      0.23     20747

    accuracy                           0.18     33918
   macro avg       0.33      0.34      0.17     33918
weighted avg       0.47      0.18      0.20     33918

Confusion matrix at  50 % :
 [[ 2229   222   407]
 [ 7852   866  1595]
 [16020  1723  3004]]


In [86]:

# 10% labelled data
start = time.time()
ml,unlabeled_set_90 = maskfunc(y, 90)
label_prop_model.fit(x, ml)
score_ninety = label_prop_model.score(x, ml)
print('\nAccuracy with 90 % unlabelled data\t',round(score_ninety,4))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf(unlabeled_set_90,90)

Total samples considered :		 67836
Count of labelled points at  90 % :	 6783
Count of unlabelled points at  90 % :	 61053

Accuracy with 90 % unlabelled data	 0.02

Metrics:

Classifcation report at  90 % :

              precision    recall  f1-score   support

           0       0.09      0.97      0.16      5485
           1       0.29      0.01      0.02     19173
           2       0.57      0.02      0.03     36395

    accuracy                           0.10     61053
   macro avg       0.32      0.33      0.07     61053
weighted avg       0.44      0.10      0.04     61053

Confusion matrix at  90 % :
 [[ 5297    81   107]
 [18635   204   334]
 [35392   419   584]]


In [87]:

# 5% labelled data
start = time.time()
ml,unlabeled_set_95 = maskfunc(y, 95)
label_prop_model.fit(x, ml)
score_ninefive = label_prop_model.score(x, ml)
print('\nAccuracy with 95 % unlabelled data\t',round(score_ninefive,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf(unlabeled_set_95,95)

Total samples considered :		 67836
Count of labelled points at  95 % :	 3391
Count of unlabelled points at  95 % :	 64445

Accuracy with 95 % unlabelled data	 0.0072

Metrics:

Classifcation report at  95 % :

              precision    recall  f1-score   support

           0       0.09      0.99      0.16      5805
           1       0.33      0.00      0.01     20408
           2       0.57      0.01      0.01     38232

    accuracy                           0.09     64445
   macro avg       0.33      0.33      0.06     64445
weighted avg       0.45      0.09      0.03     64445

Confusion matrix at  95 % :
 [[ 5729    35    41]
 [20143   101   164]
 [37793   166   273]]


# Semisupervised learning - label spreading

In [98]:
# classification report and confusion matrix function
from sklearn.metrics import roc_curve,auc,precision_recall_curve,confusion_matrix,accuracy_score,classification_report
import time

def report_conf_spread(unlabeled_set_value, percen):
    unlabeled_set = unlabeled_set_value
    predicted_labels = label_spread_model.transduction_[unlabeled_set]
    true_labels = y[unlabeled_set]
    
    cm = confusion_matrix(true_labels, predicted_labels, labels=label_prop_model.classes_)
    print('Classifcation report at ',percen,'% :\n')
    print(classification_report(true_labels, predicted_labels))

    print('Confusion matrix at ',percen,'% :\n',cm)

In [99]:
from sklearn.semi_supervised import LabelSpreading

label_spread_model = LabelSpreading(kernel='knn',n_neighbors=8)



In [100]:
# 100% labelled data
start = time.time()
label_spread_model.fit(x, y)
sp_score_zero = label_spread_model.score(x, y)
print('\nAccuracy with 0 % unlabelled data\t',round(sp_score_zero,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')


Accuracy with 0 % unlabelled data	 0.64

Run time  :	 250.93  seconds


In [101]:
# 90% labelled data
start = time.time()
ml,unlabeled_set_10 = maskfunc(y, 10)
label_spread_model.fit(x, ml)
sp_score_ten = label_spread_model.score(x, ml)
print('\nAccuracy with 10 % unlabelled data\t',round(sp_score_ten,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf_spread(unlabeled_set_10,10)

Total samples considered :		 67836
Count of labelled points at  10 % :	 61052
Count of unlabelled points at  10 % :	 6784

Accuracy with 10 % unlabelled data	 0.58

Run time  :	 246.01  seconds

Metrics:

Classifcation report at  10 % :

              precision    recall  f1-score   support

           0       0.09      0.03      0.05       383
           1       0.23      0.34      0.27      1279
           2       0.78      0.71      0.74      5122

    accuracy                           0.60      6784
   macro avg       0.36      0.36      0.35      6784
weighted avg       0.63      0.60      0.62      6784

Confusion matrix at  10 % :
 [[  12  123  248]
 [  33  440  806]
 [  95 1377 3650]]


In [102]:
# 80% labelled data
start = time.time()
ml,unlabeled_set_20 = maskfunc(y, 20)
label_spread_model.fit(x, ml)
sp_score_twenty = label_spread_model.score(x, ml)
print('\nAccuracy with 20 % unlabelled data\t',round(sp_score_twenty,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf_spread(unlabeled_set_20,20)

Total samples considered :		 67836
Count of labelled points at  20 % :	 54268
Count of unlabelled points at  20 % :	 13568

Accuracy with 20 % unlabelled data	 0.52

Run time  :	 254.88  seconds

Metrics:

Classifcation report at  20 % :

              precision    recall  f1-score   support

           0       0.08      0.04      0.05       922
           1       0.26      0.38      0.30      3094
           2       0.72      0.65      0.68      9552

    accuracy                           0.54     13568
   macro avg       0.35      0.35      0.35     13568
weighted avg       0.57      0.54      0.55     13568

Confusion matrix at  20 % :
 [[  37  322  563]
 [ 108 1166 1820]
 [ 315 3064 6173]]


In [103]:
# 50% labelled data
start = time.time()
ml,unlabeled_set_50 = maskfunc(y, 50)
label_spread_model.fit(x, ml)
sp_score_fifty = label_spread_model.score(x, ml)
print('\nAccuracy with 50 % unlabelled data\t',round(sp_score_fifty,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf_spread(unlabeled_set_50,50)

Total samples considered :		 67836
Count of labelled points at  50 % :	 33918
Count of unlabelled points at  50 % :	 33918

Accuracy with 50 % unlabelled data	 0.34

Run time  :	 257.79  seconds

Metrics:

Classifcation report at  50 % :

              precision    recall  f1-score   support

           0       0.09      0.10      0.09      2738
           1       0.33      0.38      0.35     10374
           2       0.63      0.58      0.61     20806

    accuracy                           0.48     33918
   macro avg       0.35      0.35      0.35     33918
weighted avg       0.50      0.48      0.49     33918

Confusion matrix at  50 % :
 [[  268   982  1488]
 [  980  3908  5486]
 [ 1765  6911 12130]]


In [ ]:
# 90% labelled data
start = time.time()
ml,unlabeled_set_90 = maskfunc(y, 90)
label_spread_model.fit(x, ml)
sp_score_ninety = label_spread_model.score(x, ml)
print('\nAccuracy with 90 % unlabelled data\t',round(sp_score_ninety,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf_spread(unlabeled_set_90,90)

Total samples considered :		 67836
Count of labelled points at  90 % :	 6783
Count of unlabelled points at  90 % :	 61053


In [96]:
# 95% labelled data
start = time.time()
ml,unlabeled_set_95 = maskfunc(y, 95)
label_spread_model.fit(x, ml)
sp_score_ninetyfive = label_spread_model.score(x, ml)
print('\nAccuracy with 90 % unlabelled data\t',round(sp_score_ninetyfive,2))
end = time.time()
print('\nRun time  :\t',round(end-start, 2),' seconds')
print('\nMetrics:\n')
report_conf_spread(unlabeled_set_95,95)

Total samples considered :		 67836
Count of labelled points at  95 % :	 3391
Count of unlabelled points at  95 % :	 64445

Accuracy with 90 % unlabelled data	 0.04

Metrics:

Classifcation report at  95 % :

              precision    recall  f1-score   support

           0       0.09      0.23      0.13      5799
           1       0.32      0.33      0.33     20410
           2       0.61      0.46      0.53     38236

    accuracy                           0.40     64445
   macro avg       0.34      0.34      0.33     64445
weighted avg       0.47      0.40      0.43     64445

Confusion matrix at  95 % :
 [[ 1332  1961  2506]
 [ 4842  6834  8734]
 [ 8238 12236 17762]]


# Semisupervised learning - particle competion and cooperation

In [59]:
import numpy as np
import random
from sklearn.metrics import confusion_matrix
from pypcc import ParticleCompetitionAndCooperation



In [67]:
#RUN THE MODEL
model = ParticleCompetitionAndCooperation()

model.fit(x, y)
pred = np.array(model.predict(x))

#SEPARATE PREDICTED SAMPLES
labels = np.array(labels[y == -1]).astype(int)
pred = pred[y == -1]

#PRINT CONFUSION MATRIX
print(confusion_matrix(labels, pred))

MemoryError: Unable to allocate 17.1 GiB for an array with shape (67836, 67836) and data type int32

# Semisupervised -  Naive bayes

In [4]:
# classification report and confusion matrix function
from sklearn.metrics import roc_curve,auc,precision_recall_curve,confusion_matrix,accuracy_score,classification_report
import time

def report_conf_nb(unlabeled_set_value, percen):
    unlabeled_set = unlabeled_set_value
    predicted_labels = GNBClassifier.transduction_[unlabeled_set]
    true_labels = y[unlabeled_set]
    
    cm = confusion_matrix(true_labels, predicted_labels, labels=label_prop_model.classes_)
    print('Classifcation report at ',percen,'% :\n')
    print(classification_report(true_labels, predicted_labels))

    print('Confusion matrix at ',percen,'% :\n',cm)

In [15]:
from sklearn.naive_bayes import GaussianNB
import time
import numpy as np


# 10% labelled data
start = time.time()
GNBClassifier= GaussianNB()
ml,unlabeled_set_10 = maskfunc(y, 10)
GNBClassifier.fit(x,ml)
gnb_sp_score_10 = GNBClassifier.score(x, ml)
print('\nAccuracy with 10 % unlabelled data\t',round(gnb_sp_score_10,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')
#report_conf_nb(unlabeled_set_10,10)

Total samples considered :		 67836
Count of labelled points at  10 % :	 61052
Count of unlabelled points at  10 % :	 6784

Accuracy with 10 % unlabelled data	 0.44

Metrics:


Run time  :	 0.42  seconds


In [16]:
# 20% labelled data

ml,unlabeled_set_20 = maskfunc(y, 20)
GNBClassifier.fit(x,ml)
gnb_sp_score_20 = GNBClassifier.score(x, ml)
print('\nAccuracy with 10 % unlabelled data\t',round(gnb_sp_score_20,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  20 % :	 54268
Count of unlabelled points at  20 % :	 13568

Accuracy with 10 % unlabelled data	 0.43

Metrics:


Run time  :	 59.27  seconds


In [17]:
# 50% labelled data

ml,unlabeled_set_50 = maskfunc(y, 50)
GNBClassifier.fit(x,ml)
gnb_sp_score_50 = GNBClassifier.score(x, ml)
print('\nAccuracy with 50 % unlabelled data\t',round(gnb_sp_score_50,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  50 % :	 33918
Count of unlabelled points at  50 % :	 33918

Accuracy with 10 % unlabelled data	 0.55

Metrics:


Run time  :	 94.39  seconds


In [18]:
# 90% labelled data

ml,unlabeled_set_90 = maskfunc(y, 90)
GNBClassifier.fit(x,ml)
gnb_sp_score_90 = GNBClassifier.score(x, ml)
print('\nAccuracy with 90 % unlabelled data\t',round(gnb_sp_score_90,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  90 % :	 6783
Count of unlabelled points at  90 % :	 61053

Accuracy with 90 % unlabelled data	 0.77

Metrics:


Run time  :	 126.37  seconds


In [19]:
# 50% labelled data

ml,unlabeled_set_95 = maskfunc(y, 95)
GNBClassifier.fit(x,ml)
gnb_sp_score_95 = GNBClassifier.score(x, ml)
print('\nAccuracy with 95 % unlabelled data\t',round(gnb_sp_score_95,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  95 % :	 3391
Count of unlabelled points at  95 % :	 64445

Accuracy with 95 % unlabelled data	 0.83

Metrics:


Run time  :	 156.01  seconds


# Semi supervised - ML-KNN

In [11]:
from pomegranate import *
d1 = IndependentComponentsDistribution([NormalDistribution(5, 2), NormalDistribution(6, 1), NormalDistribution(9, 1)])
d2 = IndependentComponentsDistribution([NormalDistribution(2, 1), NormalDistribution(8, 1), NormalDistribution(5, 1)])
d3 = IndependentComponentsDistribution([NormalDistribution(3, 1), NormalDistribution(5, 3), NormalDistribution(4, 1)])
model = NaiveBayes([d1, d2, d3])

In [ ]:
model = NaiveBayes.from_samples(NormalDistribution, x, y)

# Semi supervised - KNN

In [12]:
# classification report and confusion matrix function
from sklearn.metrics import roc_curve,auc,precision_recall_curve,confusion_matrix,accuracy_score,classification_report
import numpy as np
def report_knn_conf(unlabeled_set_value, percen):
    unlabeled_set = unlabeled_set_value
    predicted_labels = KNNClassifier.transduction_[unlabeled_set]
    true_labels = y[unlabeled_set]
    
    cm = confusion_matrix(true_labels, predicted_labels, labels=label_prop_model.classes_)
    print('Classifcation report at ',percen,'% :\n')
    print(classification_report(true_labels, predicted_labels))

    print('Confusion matrix at ',percen,'% :\n',cm)

In [14]:
# KNN classifier
import time
from sklearn.neighbors import KNeighborsClassifier

start = time.time()
KNNClassifier = KNeighborsClassifier(n_neighbors=8)

# 90% labelled data
ml,unlabeled_set_10 = maskfunc(y,10)
KNNClassifier.fit(x,ml)
knn_score_10 = KNNClassifier.score(x, ml)
print('\nAccuracy with 10 % unlabelled data\t',round(knn_score_10,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')



Total samples considered :		 67836
Count of labelled points at  10 % :	 61052
Count of unlabelled points at  10 % :	 6784

Accuracy with 95 % unlabelled data	 0.59

Metrics:


Run time  :	 141.24  seconds


In [15]:
# 100% labelled data
KNNClassifier.fit(x,y)
knn_score_0 = KNNClassifier.score(x, y)
print('\nAccuracy with 0 % unlabelled data\t',round(knn_score_0,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')



Accuracy with 95 % unlabelled data	 0.65

Metrics:


Run time  :	 288.46  seconds


In [16]:
# 80% labelled data
ml,unlabeled_set_20 = maskfunc(y,20)
KNNClassifier.fit(x,ml)
knn_score_20 = KNNClassifier.score(x, ml)
print('\nAccuracy with 20 % unlabelled data\t',round(knn_score_20,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  20 % :	 54268
Count of unlabelled points at  20 % :	 13568

Accuracy with 20 % unlabelled data	 0.56

Metrics:


Run time  :	 520.32  seconds


In [17]:
# 50% labelled data
ml,unlabeled_set_50 = maskfunc(y,50)
KNNClassifier.fit(x,ml)
knn_score_50 = KNNClassifier.score(x, ml)
print('\nAccuracy with 50 % unlabelled data\t',round(knn_score_50,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  50 % :	 33918
Count of unlabelled points at  50 % :	 33918

Accuracy with 50 % unlabelled data	 0.65

Metrics:


Run time  :	 766.73  seconds


In [18]:
# 10% labelled data
ml,unlabeled_set_90 = maskfunc(y,90)
KNNClassifier.fit(x,ml)
knn_score_90 = KNNClassifier.score(x, ml)
print('\nAccuracy with 10 % unlabelled data\t',round(knn_score_90,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  90 % :	 6783
Count of unlabelled points at  90 % :	 61053

Accuracy with 10 % unlabelled data	 0.9

Metrics:


Run time  :	 977.85  seconds


In [19]:
# 5% labelled data
ml,unlabeled_set_95 = maskfunc(y,95)
KNNClassifier.fit(x,ml)
knn_score_95 = KNNClassifier.score(x, ml)
print('\nAccuracy with 5 % unlabelled data\t',round(knn_score_95,2))
end = time.time()
print('\nMetrics:\n')
print('\nRun time  :\t',round(end-start, 2),' seconds')

Total samples considered :		 67836
Count of labelled points at  95 % :	 3391
Count of unlabelled points at  95 % :	 64445

Accuracy with 5 % unlabelled data	 0.95

Metrics:


Run time  :	 1189.91  seconds
